In [1]:
import pandas as pd
import minsearch

C:\Users\wifqo\Documents\PROJECTS\fitness-assistant\notebooks\minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


## Ingestion

In [2]:
df = pd.read_csv("../data.csv")

In [3]:
df

,id,exercise_name,type_of_activity,type_of_equipment,body_part,type,muscle_groups_activated,instructions
0,0,Push-Ups,Strength,Bodyweight,Upper Body,Push,"Pectorals, Triceps, Deltoids",Start in a high plank position with your hands...
1,1,Squats,Strength,Bodyweight,Lower Body,Push,"Quadriceps, Glutes, Hamstrings",Stand with feet shoulder-width apart. Lower yo...
2,2,Plank,Strength/Mobility,Bodyweight,Core,Hold,"Rectus Abdominis, Transverse Abdominis",Start in a forearm plank position with your el...
3,3,Deadlift,Strength,Barbell,Lower Body,Pull,"Glutes, Hamstrings, Lower Back","Stand with feet hip-width apart, barbell in fr..."
4,4,Bicep Curls,Strength,Dumbbells,Upper Body,Pull,"Biceps, Forearms","Stand with a dumbbell in each hand, arms fully..."
...,...,...,...,...,...,...,...,...
202,202,Incline Dumbbell Row,Strength,Dumbbells,Upper Body,Pull,"Latissimus Dorsi, Biceps",Lie face down on an incline bench with a dumbb...
203,203,Machine Lat Pulldown,Strength,Machine,Upper Body,Pull,"Latissimus Dorsi, Biceps",Sit at a lat pulldown machine with a wide grip...
204,204,One-Arm Cable Row,Strength,Cable Machine,Upper Body,Pull,"Latissimus Dorsi, Biceps",Stand facing a cable machine with the handle a...
205,205,Face Pull,Strength,Cable Machine,Upper Body,Pull,"Rear Deltoids, Trapezius, Rhomboids",Attach a rope handle to a high cable pulley. P...


In [4]:
df.columns

Index(['id', 'exercise_name', 'type_of_activity', 'type_of_equipment',
       'body_part', 'type', 'muscle_groups_activated', 'instructions'],
      dtype='object')

In [19]:
index = minsearch.Index(
    text_fields = ['exercise_name', 'type_of_activity', 'type_of_equipment',
       'body_part', 'type', 'muscle_groups_activated', 'instructions'],
    keyword_fields = ['id']
)

In [20]:
documents = df.to_dict(orient='records')

In [21]:
index.fit(documents)

In [23]:
query = 'give me exercises for hamstrings' 

## RAG Flow

In [24]:
import os
from dotenv import load_dotenv
import openai
from openai import OpenAI

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()

In [25]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )
    return results

In [26]:
prompt_template = """
    You're a coach teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: {context}
""".strip()

entry_template = """
    exercise_name: {exercise_name}
    type_of_activity: {type_of_activity}
    type_of_equipment: {type_of_equipment}
    body_part: {body_part}
    type: {type}
    muscle_groups_activated: {muscle_groups_activated}
    instructions: {instructions}
""".strip()


def build_prompt(query, search_results):
    
    context = ""

    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [27]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [28]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [29]:
answer = rag(query)
print(answer)

Here are some exercises that target the hamstrings:

1. **Leg Curl**:
   - **Activity Type**: Strength
   - **Equipment**: Machine
   - **Instructions**: Lie face down on a leg curl machine and curl the pad towards your glutes, then return to the starting position.

2. **Hamstring Stretch**:
   - **Activity Type**: Mobility
   - **Equipment**: Bodyweight
   - **Instructions**: Stand upright and place one heel on a bench or step. Lean forward slightly to feel a stretch in the hamstring of the elevated leg.

3. **Machine Leg Curl**:
   - **Activity Type**: Strength
   - **Equipment**: Machine
   - **Instructions**: Sit on a leg curl machine and curl the pad down towards your glutes, then return to the starting position.

4. **Seated Leg Curl**:
   - **Activity Type**: Strength
   - **Equipment**: Machine
   - **Instructions**: Sit on a leg curl machine and curl the pad down towards your glutes, then return to the starting position.

5. **Prone Leg Curl**:
   - **Activity Type**: Strength